In [15]:

from photutils.aperture import aperture_photometry, SkyCircularAperture
from astropy.table import Table

import photutils

from astropy.io import fits

from astropy.wcs import WCS
import astropy.units as u
from astropy.coordinates import SkyCoord


import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
from astropy.io import ascii

In [7]:
my_vandercat_complete = ascii.read('/Users/neal/Documents/PhD/projects/SED-fitting/Clump/results/last_aperturecorrcted_vander.csv').to_pandas()

# 1) Weight Maps:

In [26]:
# Load the weight maps for each band
weight_maps = {
   
    'f125w': fits.getdata('/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_wfc3_ir_f125w_060mas_v1.0_wht.fits'),
    'f140w': fits.getdata('/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_wfc3_ir_f140w_060mas_v1.0_wht.fits'),
    'f160w': fits.getdata('/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_wfc3_ir_f160w_060mas_v1.0_wht.fits'),
    'f275w': fits.getdata('/Users/neal/Documents/PhD/projects/SED-fitting/Clump/UV candels/hlsp_uvcandels_hst_wfc3-uvis_goodss-60mas_f275w_v1.0_wht.fits'),
    'f435w': fits.getdata('/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f435w_060mas_v1.5_wht.fits'),
    'f606w': fits.getdata('/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f606w_060mas_v1.5_wht.fits'),
    'f775w': fits.getdata('/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f775w_060mas_v1.5_wht.fits'),
    'f850w': fits.getdata('/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f850l_060mas_v1.5_wht.fits'),
   
}

In [34]:
# Load your clump catalog
clump_catalog_path = '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/results/last_aperturecorrcted_vander.csv'
clump_catalog = ascii.read(clump_catalog_path).to_pandas()

# Define aperture radius here
aperture_radius = 0.18  #in arcseconds

# Define the band suffix to WCS file mapping 
band_to_wcs_file = {
    '125': 'goodss_all_wfc3_ir_f125w_060mas_v1.0_wht.fits',
    '140': 'goodss_all_wfc3_ir_f140w_060mas_v1.0_wht.fits',
    '160': 'goodss_all_wfc3_ir_f160w_060mas_v1.0_wht.fits',
    '275': 'hlsp_uvcandels_hst_wfc3-uvis_goodss-60mas_f275w_v1.0_wht.fits',
    '435': 'goodss_all_acs_wfc_f435w_060mas_v1.5_wht.fits',
    '606': 'goodss_all_acs_wfc_f606w_060mas_v1.5_wht.fits',
    '775': 'goodss_all_acs_wfc_f775w_060mas_v1.5_wht.fits',
    '850': 'goodss_all_acs_wfc_f850l_060mas_v1.5_wht.fits',
    
      
}

bands = ['275','435', '606', '775', '850', '125', '140', '160']
# Calculate uncertainty
def calculate_uncertainty(clump_row, wcs, weight_map, aperture_radius):

    # Convert RA and DEC to pixel coordinates
    x, y = wcs.all_world2pix(clump_row['RA_clump'], clump_row['DEC_clump'], 0)
    #  Create a SkyCoord object representing the center of the aperture
    aperture_center = SkyCoord(clump_row['RA_clump'], clump_row['DEC_clump'], unit='deg')
    
    aperture = SkyCircularAperture(aperture_center, r=aperture_radius* u.arcsec)
    photometry_data = aperture_photometry(weight_map, aperture , wcs=wcs)
    sum_weights = np.sum(photometry_data['aperture_sum'])

    # Calculate uncertainty
    if sum_weights == 0:
        uncertainty = np.nan  # Handle divide by zero
    else:
        uncertainty = np.sqrt(1 / sum_weights)
    
    return uncertainty
    # uncertainty = np.sqrt(1 / sum_weights)
    # return uncertainty

# Initialize a dictionary to store the uncertainties for each band
uncertainties = {band: [] for band in bands}

# Loop over each clump in catalog
for index, clump_row in clump_catalog.iterrows():
    if clump_row['DEC_clump'] == -99.0:
        # If invalid coordinates, append NaN for all bands and continue to the next iteration
        for band_suffix in bands:
            uncertainties[band_suffix].append(np.nan)
        print(f"Skipping clump due to invalid coordinates: {clump_row['CANDELS_ID']}, {clump_row['Clump_ID']}")
        continue
    else:
        # Create SkyCoord object from clump coordinates
        clump_coords = SkyCoord(ra=clump_row['RA_clump']*u.deg, dec=clump_row['DEC_clump']*u.deg)
    
    for band_suffix in bands:
        # Get the weight map for the current band
        weight_map = fits.getdata('/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/' + band_to_wcs_file[band_suffix])
        
        # Extract the WCS for the current band from the weight map FITS file
        with fits.open('/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/' + band_to_wcs_file[band_suffix]) as hdul:
            wcs = WCS(hdul[0].header)
            wcs.sip = None  # Disable any SIP distortion if present
        
        # Calculate the uncertainty for this clump and band
        uncertainty = calculate_uncertainty(clump_row, wcs, weight_map, aperture_radius)
        
        # Store the calculated uncertainty
        uncertainties[band_suffix].append(uncertainty)

# Add the uncertainties to the clump catalog DataFrame
for band_suffix in bands:
  

    clump_catalog[f'Uncertainty_{band_suffix}_mjy'] = uncertainties[band_suffix]

# Save the updated catalog with uncertainties
clump_catalog.to_csv('/Users/neal/Documents/PhD/projects/SED-fitting/Clump/results/uncertainty__included_clump_catalog_my_vander.csv', index=False)

Skipping clump due to invalid coordinates: 135, -99
Skipping clump due to invalid coordinates: 137, -99
Skipping clump due to invalid coordinates: 145, -99
Skipping clump due to invalid coordinates: 544, -99
Skipping clump due to invalid coordinates: 583, -99
Skipping clump due to invalid coordinates: 606, -99
Skipping clump due to invalid coordinates: 675, -99
Skipping clump due to invalid coordinates: 733, -99
Skipping clump due to invalid coordinates: 835, -99
Skipping clump due to invalid coordinates: 879, -99
Skipping clump due to invalid coordinates: 946, -99
Skipping clump due to invalid coordinates: 986, -99
Skipping clump due to invalid coordinates: 1132, -99
Skipping clump due to invalid coordinates: 1183, -99
Skipping clump due to invalid coordinates: 1187, -99
Skipping clump due to invalid coordinates: 1198, -99
Skipping clump due to invalid coordinates: 1203, -99
Skipping clump due to invalid coordinates: 1208, -99
Skipping clump due to invalid coordinates: 1485, -99
Skipp

# Poisson noise:

In [35]:
band_to_science_file = {
    '125': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_wfc3_ir_f125w_060mas_v1.0_drz.fits',
    '140': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_wfc3_ir_f140w_060mas_v1.0_drz.fits',
    '160': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_wfc3_ir_f160w_060mas_v1.0_drz.fits',
    '435': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f435w_060mas_v1.5_drz.fits',
    '606': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f606w_060mas_v1.5_drz.fits',
    '775': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f775w_060mas_v1.5_drz.fits',
    '850': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f850l_060mas_v1.5_drz.fits',


}

In [36]:
band_to_exptime_file = {
    '125': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_wfc3_ir_f125w_060mas_v1.0_exp.fits',
    '140': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_wfc3_ir_f140w_060mas_v1.0_exp.fits',
    '160': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_wfc3_ir_f160w_060mas_v1.0_exp.fits',
    '435': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f435w_060mas_v1.5_exp.fits',
    '606': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f606w_060mas_v1.5_exp.fits',
    '775': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f775w_060mas_v1.5_exp.fits',
    '850': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f850l_060mas_v1.5_exp.fits',

}

In [1]:
band_to_files = {
    '125': {
        'science': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_wfc3_ir_f125w_060mas_v1.0_drz.fits',
        'exptime': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_wfc3_ir_f125w_060mas_v1.0_exp.fits'
    },
    '140': {
        'science': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_wfc3_ir_f140w_060mas_v1.0_drz.fits',
        'exptime': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_wfc3_ir_f140w_060mas_v1.0_exp.fits'
    },
    '160': {
        'science': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_wfc3_ir_f160w_060mas_v1.0_drz.fits',
        'exptime': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_wfc3_ir_f160w_060mas_v1.0_exp.fits'
    },
    '435': {
        'science': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f435w_060mas_v1.5_drz.fits',
        'exptime': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f435w_060mas_v1.5_exp.fits'
    },
    '606': {
        'science': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f606w_060mas_v1.5_drz.fits',
        'exptime': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f606w_060mas_v1.5_exp.fits'
    },
    '775': {
        'science': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f775w_060mas_v1.5_drz.fits',
        'exptime': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f775w_060mas_v1.5_exp.fits'
    },
    '850': {
        'science': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f850l_060mas_v1.5_drz.fits',
        'exptime': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f850l_060mas_v1.5_exp.fits'
    },
}

In [2]:
#  Function to calculate uncertainty
def calculate_poisson_noise(clump_row, wcs, science_image, exptime_image, aperture_radius):
    # Convert RA and DEC to pixel coordinates
    x, y = wcs.all_world2pix(clump_row['RA_clump'], clump_row['DEC_clump'], 0)
    # Create a SkyCoord object representing the center of the aperture
    aperture_center = SkyCoord(clump_row['RA_clump'], clump_row['DEC_clump'], unit='deg')
    aperture = SkyCircularAperture(aperture_center, r=aperture_radius * u.arcsec)
    
    # Calculate counts (science image x exposure time image)
    counts_image = science_image * exptime_image
    
    # Perform aperture photometry to sum the counts within the aperture
    photometry_data = aperture_photometry(counts_image, aperture, wcs=wcs)
    counts_sum = photometry_data['aperture_sum'][0]
    
    # Calculate Poisson noise as the square root of the counts sum
    poisson_noise = np.sqrt(counts_sum) / np.mean(exptime_image)
    
    return poisson_noise


In [3]:
# Initialize a dictionary to store the Poisson uncertainties for each band
poisson_uncertainties = {band: [] for band in band_to_files.keys()}

In [5]:

from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy.table import Table
import pandas as pd
import numpy as np
from photutils import SkyCircularAperture, aperture_photometry
import astropy.units as u


/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_73894/658513016.py:7: DeprecationWarning: `photutils.SkyCircularAperture` is a deprecated alias for `photutils.aperture.SkyCircularAperture` and will be removed in the future. Instead, please use `from photutils.aperture import SkyCircularAperture` to silence this warning.
  from photutils import SkyCircularAperture, aperture_photometry
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_73894/658513016.py:7: DeprecationWarning: `photutils.aperture_photometry` is a deprecated alias for `photutils.aperture.aperture_photometry` and will be removed in the future. Instead, please use `from photutils.aperture import aperture_photometry` to silence this warning.
  from photutils import SkyCircularAperture, aperture_photometry


In [7]:
bands = ['435', '606', '775', '850', '125', '140', '160']

In [9]:
clump_catalog_path = '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/results/last_aperturecorrcted_vander.csv'
clump_catalog = Table.read(clump_catalog_path).to_pandas()

In [6]:
# Load all the FITS data outside the loop
fits_data = {}
for band, file_paths in band_to_files.items():
    with fits.open(file_paths['science']) as hdul_science:
        wcs = WCS(hdul_science[0].header)
        science_image = hdul_science[0].data
    with fits.open(file_paths['exptime']) as hdul_exptime:
        exptime_image = hdul_exptime[0].data
    fits_data[band] = (wcs, science_image, exptime_image)


In [11]:
# Define aperture radius here
aperture_radius = 0.18  # in arcseconds

In [12]:
#  Loop over each clump in the catalog
for index, clump_row in clump_catalog.iterrows():
    for band in band_to_files.keys():
        wcs, science_image, exptime_image = fits_data[band]
        poisson_noise = calculate_poisson_noise(clump_row, wcs, science_image, exptime_image, aperture_radius)
        poisson_uncertainties[band].append(poisson_noise)


# Add the Poisson uncertainties to the clump catalog DataFrame
for band in bands:
    clump_catalog[f'Poisson_Uncertainty_{band}'] = poisson_uncertainties[band]

# Save the updated catalog with Poisson uncertainties
clump_catalog.to_csv('/Users/neal/Documents/PhD/projects/SED-fitting/Clump/results/2nd_poisson_uncertainty_included_clump_catalog.csv', index=False)
